<a href="https://colab.research.google.com/github/Anu589/Satellite-EuroSAT/blob/main/1st%20version%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install patchify

In [2]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import torch
import torchvision
import matplotlib.pyplot as plt
from PIL import Image
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from torchvision.io import read_image
import torchvision.transforms as T
import random
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets list

ref                                                                    title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
atharvasoundankar/chocolate-sales                                      Chocolate Sales Data 📊🍫                                 14473  2025-03-19 03:51:40.270000           8852        144  1.0              
abdulmalik1518/mobiles-dataset-2025                                    Mobiles Dataset (2025)                                  20314  2025-02-18 06:50:24.370000          15193        268  1.0              
mahmoudelhemaly/students-grading-dataset                               Student Performance & Behavior Dataset                 520428  2025-02-17 17:38:46.653000          10896 

In [5]:
!kaggle datasets download -d apollo2506/eurosat-dataset -p data/ --unzip

Dataset URL: https://www.kaggle.com/datasets/apollo2506/eurosat-dataset
License(s): CC0-1.0


In [6]:
# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [7]:
# Set directories
data_dir = "data/EuroSAT"

In [8]:
if os.path.exists(data_dir):
    classes = os.listdir(data_dir)
    print(f" Dataset contains {len(classes)} categories:")
    print(classes)
else:
    print(" Dataset extraction failed. Check the ZIP file.")


 Dataset contains 14 categories:
['test.csv', 'Pasture', 'validation.csv', 'PermanentCrop', 'HerbaceousVegetation', 'Forest', 'Residential', 'Highway', 'label_map.json', 'River', 'SeaLake', 'Industrial', 'train.csv', 'AnnualCrop']


In [9]:
# Define image size and number of classes
IMG_SIZE = (256, 256)
NUM_CLASSES = 10  # Update based on dataset

In [10]:
# Define color mapping for each class
land_classes = {
    "AnnualCrop": (255, 255, 0),     # Yellow
    "Forest": (0, 255, 0),           # Green
    "HerbaceousVegetation": (144, 238, 144),  # Light Green
    "Highway": (128, 128, 128),      # Gray
    "Industrial": (255, 0, 0),       # Red
    "Pasture": (139, 69, 19),        # Brown
    "PermanentCrop": (0, 128, 0),    # Dark Green
    "Residential": (255, 165, 0),    # Orange
    "River": (0, 0, 255),            # Blue
    "SeaLake": (0, 255, 255)         # Cyan
}

In [11]:
# Function to generate mask
def generate_mask(image):
    mask = np.zeros_like(image)

    for class_name, color in land_classes.items():
        # Define custom rules for land segmentation (example: color-based detection)
        if class_name == "Forest":
            lower, upper = np.array([0, 50, 0]), np.array([100, 255, 100])  # Detect green
        elif class_name == "Highway":
            lower, upper = np.array([100, 100, 100]), np.array([200, 200, 200])  # Detect gray
        elif class_name in ["River", "SeaLake"]:
            lower, upper = np.array([0, 0, 100]), np.array([100, 100, 255])  # Detect blue water
        else:
            continue  # Other classes require advanced methods

        # Create binary mask
        feature_mask = cv2.inRange(image, lower, upper)
        mask[feature_mask > 0] = color  # Apply color

    return mask

In [12]:
output_root = "masked_eurosat_images"
os.makedirs(output_root, exist_ok=True)


for path, subdirs, folders in os.walk(data_dir):
    dir_name = path.split(os.path.sep)[-1]
    print(f"Found directory: {dir_name}")

    output_dir = os.path.join(output_root, dir_name)
    os.makedirs(output_dir, exist_ok=True)

    images = os.listdir(path)
    for i, img_name in enumerate(images):
        if img_name.endswith(".jpg"):
            img_path = os.path.join(path, img_name)
            image = cv2.imread(img_path)
            if image is not None:  # Check if image was read successfully
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                mask = generate_mask(image)
                masked_image = cv2.addWeighted(image, 0.6, mask, 0.4, 0)

                output_path = os.path.join(output_dir, img_name)
                cv2.imwrite(output_path, cv2.cvtColor(masked_image, cv2.COLOR_RGB2BGR))
                print(f"Saved masked image: {output_path}")
            else:
                print(f"Error reading image: {img_path}")


Streaming output truncated to the last 5000 lines.
Saved masked image: masked_eurosat_images/Industrial/Industrial_1567.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_1053.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_562.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_529.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_1794.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_1455.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_2055.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_1513.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_505.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_461.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_1775.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_2361.jpg
Saved masked image: masked_eurosat_images/Industrial/Industrial_1799.jpg
Save

In [27]:
train_csv = pd.read_csv("/content/data/EuroSAT/train.csv")
val_csv = pd.read_csv("/content/data/EuroSAT/validation.csv")
test_csv = pd.read_csv("/content/data/EuroSAT/test.csv")

In [28]:
def load_image_paths(csv_file, data_dir):
    image_paths = []
    labels = []

    for _, row in csv_file.iterrows():
        img_filename = row["Filename"].strip()  # Get full filename
        category = img_filename.split('/')[0]  # Extract category (e.g., "AnnualCrop")

        if category not in category_labels:
            print(f"Warning: Category '{category}' not found in labels.json")
            continue  # Skip files with missing labels

        img_path = os.path.join(data_dir, img_filename)  # Get image path
        image_paths.append(img_path)
        labels.append(category_labels[category])  # Get category label

    return image_paths, labels

In [29]:
from tensorflow.keras.utils import Sequence
import numpy as np

class DataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size, img_size, num_classes, shuffle=True):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        images = []
        labels = []
        for img_path, label in zip(batch_image_paths, batch_labels):
            # Load and preprocess image
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.img_size)
            img = img / 255.0  # Normalize

            # Convert label to one-hot encoding
            label = tf.keras.utils.to_categorical(label, num_classes=self.num_classes)

            images.append(img)
            labels.append(label)

        return np.array(images), np.array(labels)

    def on_epoch_end(self):
        if self.shuffle:
            indices = np.arange(len(self.image_paths))
            np.random.shuffle(indices)
            self.image_paths = [self.image_paths[i] for i in indices]
            self.labels = [self.labels[i] for i in indices]

In [30]:
# Load image paths and labels
train_image_paths, y_train = load_image_paths(train_csv, data_dir)
val_image_paths, y_val = load_image_paths(val_csv, data_dir)
test_image_paths, y_test = load_image_paths(test_csv, data_dir)

In [31]:
# Define parameters
batch_size = 16  # Adjust based on memory availability
img_size = (128, 128)  # Resize images to reduce memory usage
num_classes = 10  # Number of classes in EuroSAT

# Create data generators
train_generator = DataGenerator(train_image_paths, y_train, batch_size, img_size, num_classes)
val_generator = DataGenerator(val_image_paths, y_val, batch_size, img_size, num_classes)
test_generator = DataGenerator(test_image_paths, y_test, batch_size, img_size, num_classes)

In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam

In [34]:
def create_model(input_shape=(128, 128, 3), num_classes=10):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')  # Output layer for classification
    ])
    return model

# Create the model
model = create_model()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=10
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 441s 370ms/step - accuracy: 0.3387 - loss: 1.7486 - val_accuracy: 0.5459 - val_loss: 1.2776
Epoch 2/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 435s 365ms/step - accuracy: 0.5562 - loss: 1.2339 - val_accuracy: 0.6041 - val_loss: 1.0978
Epoch 3/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 436s 360ms/step - accuracy: 0.6203 - loss: 1.0715 - val_accuracy: 0.6600 - val_loss: 0.9865
Epoch 4/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 420s 355ms/step - accuracy: 0.6690 - loss: 0.9455 - val_accuracy: 0.6709 - val_loss: 0.9023
Epoch 5/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 434s 349ms/step - accuracy: 0.6974 - loss: 0.8572 - val_accuracy: 0.7013 - val_loss: 0.8276
Epoch 6/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 422s 357ms/step - accuracy: 0.7162 - loss: 0.8181 - val_accuracy: 0.6878 - val_loss: 0.8587
Epoch 7/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 441s 373ms/step - accuracy: 0.7381 - loss: 0.7473 - val_accuracy: 0.7339 - val_loss: 0.7555
Epoch 8/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 485s 409ms/step - ac

In [37]:
# Save model
model.save("eurosat_unet.h5")

print("Model training complete and saved as eurosat_unet.h5")

Model training complete and saved as eurosat_unet.h5
